### **Installing Pytorch Lightning**

In [ ]:
!pip install pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 56.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink

In [ ]:
# Mount Google Drive and unzip the dataset
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

# Unzip dataset
zip_path = '/content/drive/MyDrive/inaturalist_12K.zip'
extract_path = '/content/inaturalist_12K'

if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/')

Mounted at /content/drive


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset
import wandb
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm

# Define a dictionary to map activation function names to their corresponding PyTorch classes
activation_map = {
    'ReLU': nn.ReLU,
    'GELU': nn.GELU,
    'SiLU': nn.SiLU,
    'Mish': nn.Mish
}

# Flexible Convolutional Neural Network class definition
class FlexibleCNN(nn.Module):
    def __init__(self, conv_filters, kernel_sizes, activation_fn_str, dense_units, dropout_rate=0.5, num_classes=10):
        super(FlexibleCNN, self).__init__()

        # Validate activation function string
        if activation_fn_str not in activation_map:
            raise ValueError(f"Unsupported activation function: {activation_fn_str}")
        activation_fn = activation_map[activation_fn_str]

        self.convs = nn.ModuleList()
        in_channels = 3  # RGB channels for images

        # Create convolutional blocks with Conv2D -> Activation -> MaxPooling
        for filters, k in zip(conv_filters, kernel_sizes):
            self.convs.append(nn.Sequential(
                nn.Conv2d(in_channels, filters, kernel_size=k, padding=k // 2),
                activation_fn(),  # Activation function instance
                nn.MaxPool2d(kernel_size=2)
            ))
            in_channels = filters  # Update for next layer

        self.flatten = nn.Flatten()

        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(dense_units, num_classes)

    def forward(self, x):
        for conv in self.convs:
            x = conv(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Utility function to calculate accuracy
def accuracy(preds, labels):
    _, pred_classes = preds.max(1)
    correct = (pred_classes == labels).sum().item()
    return correct / len(labels)

# Load training and validation data using stratified split
def load_data(data_dir, batch_size, val_split=0.2):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    full_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
    targets = [label for _, label in full_dataset.imgs]

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=val_split, random_state=42)
    train_idx, val_idx = next(splitter.split(full_dataset.imgs, targets))

    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

# Load test data
def load_test_data(data_dir, batch_size):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return test_loader, test_dataset

# Main training function integrated with Weights & Biases
def train_model(config=None):
    wandb.init(config=config)
    config = wandb.config

    # Apply data augmentation only if specified
    if config.data_augmentation == 'yes':
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ])
    else:
        train_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])

    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # Select device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize model
    model = FlexibleCNN(
        conv_filters=config.conv_filters,
        kernel_sizes=config.kernel_sizes,
        activation_fn_str=config.activation_fn,
        dense_units=config.dense_units,
        dropout_rate=config.dropout,
        num_classes=10
    ).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

    # Load data loaders
    train_loader, val_loader = load_data(config.data_dir, config.batch_size)

    # Training loop
    for epoch in range(config.epochs):
        model.train()
        total_train_loss, total_train_acc = 0, 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
            inputs, labels = batch[0].to(device), batch[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            total_train_acc += accuracy(outputs, labels)

        # Validation loop
        model.eval()
        total_val_loss, total_val_acc = 0, 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
                inputs, labels = batch[0].to(device), batch[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                total_val_acc += accuracy(outputs, labels)

        # Compute average metrics
        train_acc = total_train_acc / len(train_loader)
        val_acc = total_val_acc / len(val_loader)

        # Log metrics to Weights & Biases
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': total_train_loss / len(train_loader),
            'train_accuracy': train_acc,
            'val_loss': total_val_loss / len(val_loader),
            'val_accuracy': val_acc
        })


**Sweep Configuration**

In [ ]:
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'activation_fn': {
            'values': ['GELU', 'SiLU']
        },
        'batch_size': {
            'values': [32, 64]
        },
        'conv_filters': {
            'values': [[64, 64, 64, 64, 64]]
        },
        'kernel_sizes': {
            'values': [[3, 3, 3, 3, 3]]
        },
        'dense_units': {
            'values': [128, 256, 512]
        },
        'dropout': {
            'values': [0.3, 0.5]
        },
        'lr': {
            'distribution': 'uniform',
            'min': 0.0003,
            'max': 0.0007
        },
        'epochs': {
            'value': 10
        },
        'data_dir': {
            'value': '/content/inaturalist_12K'
        },
        'data_augmentation': {
            'values': ['yes', 'no']
        }
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="A2")
wandb.agent(sweep_id, function=train_model, count=20)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 65nw10d9
Sweep URL: https://wandb.ai/me21b164-indian-institute-of-technology-madras/A2/sweeps/65nw10d9


wandb: Agent Starting Run: 1walwjo9 with config:
wandb: 	activation_fn: SiLU
wandb: 	batch_size: 32
wandb: 	conv_filters: [64, 64, 64, 64, 64]
wandb: 	data_augmentation: yes
wandb: 	data_dir: /content/inaturalist_12K
wandb: 	dense_units: 256
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.0006120471159227737
wandb: Currently logged in as: me21b164 (me21b164-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 10 [Val]: 100%|██████████| 32/32 [00:09<00:00,  3.43it/s]


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▃▄▅▆▆▇▇█
train_loss,█▇▆▅▅▄▄▃▂▁
val_accuracy,▁▃▅▅▆▇█▇█▇
val_loss,█▆▅▃▃▂▁▂▂▅
epoch,10
train_accuracy,0.45511
train_loss,1.55861
val_accuracy,0.33496
val_loss,2.04728


...

wandb: Agent Starting Run: m2zb5do0 with config:
wandb: 	activation_fn: GELU
wandb: 	batch_size: 64
wandb: 	conv_filters: [64, 64, 64, 64, 64]
wandb: 	data_augmentation: no
wandb: 	data_dir: /content/inaturalist_12K
wandb: 	dense_units: 512
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.0005218821522417687


Epoch 10 [Val]: 100%|██████████| 16/16 [00:08<00:00,  1.82it/s]


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▃▄▅▆▆▇▇█
train_loss,█▇▆▅▄▄▃▃▂▁
val_accuracy,▁▃▄▅▅▇▆▇█▇
val_loss,█▆▅▄▃▂▁▁▂▂
epoch,10
train_accuracy,0.43172
train_loss,1.61158
val_accuracy,0.36543
val_loss,1.87764


wandb: Agent Starting Run: 14wao7y8 with config:
wandb: 	activation_fn: SiLU
wandb: 	batch_size: 32
wandb: 	conv_filters: [64, 64, 64, 64, 64]
wandb: 	data_augmentation: no
wandb: 	data_dir: /content/inaturalist_12K
wandb: 	dense_units: 256
wandb: 	dropout: 0.5
wandb: 	epochs: 10
wandb: 	kernel_sizes: [3, 3, 3, 3, 3]
wandb: 	lr: 0.0006717255301094951


Epoch 6 [Train]:  34%|███▍      | 43/125 [00:14<00:33,  2.44it/s]

## **Best Model**

Selecting configuration of the best performing model to report accuracy of test data set

In [24]:
import wandb

api = wandb.Api()
sweep = api.sweep('me21b164-indian-institute-of-technology-madras/A2/sweeps/65nw10d9')
best_run = max(sweep.runs, key=lambda run: run.summary.get('val_accuracy', 0))
best_config = best_run.config


def load_data_with_test(data_dir, batch_size, val_split=0.2):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    full_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
    targets = [label for _, label in full_dataset.imgs]

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=val_split, random_state=42)
    train_idx, val_idx = next(splitter.split(full_dataset.imgs, targets))

    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, test_dataset.classes


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def evaluate_on_test(config):


    model = FlexibleCNN(
        conv_filters=config['conv_filters'],
        kernel_sizes=config['kernel_sizes'],
        activation_fn_str=config['activation_fn'],
        dense_units=config['dense_units'],
        dropout_rate=config['dropout'],
        num_classes=10
    ).to(device)

    _, _, test_loader, class_names = load_data_with_test(config['data_dir'], config['batch_size'])

    model.eval()
    total_test_acc = 0
    predictions = []
    images = []
    labels = []

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = batch[0].to(device), batch[1].to(device)
            outputs = model(inputs)
            total_test_acc += accuracy(outputs, targets)

            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            images.extend(inputs.cpu())
            labels.extend(targets.cpu().numpy())

    test_acc = total_test_acc / len(test_loader)
    print(f"Test Accuracy: {test_acc:.4f}")

    return test_acc, images, predictions, labels, class_names


test_acc, test_images, test_preds, test_labels, class_names = evaluate_on_test(best_config)

Test Accuracy: 0.0977


## **Display (10x3) Grid**

In [ ]:
import os
import torch
import random
import matplotlib.pyplot as plt
from PIL import Image
import wandb

transform = transforms.Compose([
    transforms.Resize((256,256)),

    transforms.ToTensor(),

    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

# Define class labels
labels = ["Amphibia", "Animalia", "Arachnida", "Aves", "Fungi",
          "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]

# Device selection
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Best Model Configuration
model = FlexibleCNN(
    conv_filters=[64, 64, 64, 64, 64],
    kernel_sizes=[3, 3, 3, 3, 3],
    activation_fn_str='GELU',
    dense_units=256,
    dropout_rate=0.3,
    num_classes=10
).to(device)


# Transform for resizing, converting to tensor, and normalizing
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create a 10x3 grid for plotting
fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(8, 16))
j = -1

# Loop through the grid and plot images and predictions
for i, ax in enumerate(axes.flat):
    if i % 3 == 0:
        j += 1

    # Load a random image from the respective class folder
    folder_path = f"/content/inaturalist_12K/train/{labels[j]}"
    file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    random_file = random.choice(file_list)
    image_path = os.path.join(folder_path, random_file)

    # Load and transform the image
    image = Image.open(image_path).convert("RGB")
    image_t = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    # Get model prediction
    with torch.no_grad():
        output = model(image_t)
        probs = torch.sigmoid(output).cpu().squeeze()  # Sigmoid for multi-label output

    # Set a threshold for predicting labels (e.g., 0.5)
    threshold = 0.5
    predicted_labels = [labels[idx] for idx, prob in enumerate(probs) if prob >= threshold]
    predicted_text = ", ".join(predicted_labels) if predicted_labels else "None"

    # Plot the image
    ax.imshow(image)
    ax.axis("off")
    ax.set_ylabel(f'Actual: {labels[j]}', fontsize=9)
    ax.set_title(f'Predicted: {predicted_text}', fontsize=8)

# Adjust layout and display the plot
plt.tight_layout()
wandb.log({"Prediction Grid": wandb.Image(plt)})  # Log to wandb